In this demo we show how to implement a multipolar polarizable potential with fluctuating charges.
First, we will use the DMFF API to create a multipolar polarizable potential with fixed atomic charges:

Read input pdb and xml files, create the Hamiltonian object H.

In [4]:
import sys
import jax
import jax.numpy as jnp
import openmm.app as app
import openmm.unit as unit
from dmff.api import Hamiltonian

H = Hamiltonian('forcefield.xml')
app.Topology.loadBondDefinitions("residues.xml")
pdb = app.PDBFile("water_dimer.pdb")
rc = 4

For each force component, the Hamiltonian will create a generator. Here, we have two components: dispersion force and multipolar polarizable pme force. We will focus on the pme force. In the generator, you can find the force field parameters (charges, multipoles) stored in generator.params:

In [5]:
disp_generator, pme_generator = H.getGenerators()
print(pme_generator.params.keys())

dict_keys(['mScales', 'pScales', 'dScales', 'Q_local', 'pol', 'tholes'])


Then use the `createPotential` function to create potential functions. When `createPotentials` is invoked, it will use the topology information of the system to create potential functions, and creates many useful information (e.g.,  local axis definitions) as well. This information is stored in generator.pme_force and will be used later

In [6]:
potentials = H.createPotential(pdb.topology, nonbondedCutoff=rc*unit.angstrom, steps_pol=5)
pot_disp = potentials[-2]
pot_pme = potentials[-1]

print(pme_generator.pme_force.lpol)
print(pme_generator.pme_force.axis_type)
print(pme_generator.pme_force.axis_indices)
print(pme_generator.pme_force.steps_pol)

True
[1 0 0 1 0 0]
[[ 1  2 -1]
 [ 0  2 -1]
 [ 0  1 -1]
 [ 4  5 -1]
 [ 3  5 -1]
 [ 3  4 -1]]
None


What `pot_pme` actually does is to take force field parameters (i.e., oxygen and hydrogen charges), and expand it into atomic parameterss (i.e., charge of each atom), then calls the real pme kernel function to evaluate energy.

Therefore, the input interface of `pot_pme` is not flexible enough to specify atom-specific charges, which is what we intend to do in this demo. So we will disregard `pot_pme`, but to wrap the real pme kernel directly in a way different to `pot_pme`. The pme kernel function can be accessed via generator:

In [7]:
help(pme_generator.pme_force.get_energy)

Help on function get_energy in module dmff.admp.pme:

get_energy(positions, box, pairs, Q_local, pol, tholes, mScales, pScales, dScales, U_init=DeviceArray([[0., 0., 0.],
             [0., 0., 0.],
             [0., 0., 0.],
             [0., 0., 0.],
             [0., 0., 0.],
             [0., 0., 0.]], dtype=float32))
    # this is the wrapper that include a Uind optimizer



Let's wrap it using the following function:

In [8]:
from run import compute_leading_terms

def generate_calculator(get_energy_pme, pme_generator):
    def admp_calculator(positions, box, pairs):
        c0, c6_list = compute_leading_terms(positions,box) # compute fluctuated charges
        Q_local = pme_generator.params["Q_local"][pme_generator.map_atomtype]
        Q_local = Q_local.at[:,0].set(c0)  # change fixed charge into fluctuated one
        pol = pme_generator.params["pol"][pme_generator.map_atomtype]
        tholes = pme_generator.params["tholes"][pme_generator.map_atomtype]
        mScales = pme_generator.params["mScales"]
        pScales = pme_generator.params["pScales"]
        dScales = pme_generator.params["dScales"]
        E_pme = pme_generator.pme_force.get_energy(
            positions, 
            box, 
            pairs, 
            Q_local, 
            pol, 
            tholes, 
            mScales, 
            pScales, 
            dScales
            )
        return E_pme 
    return admp_calculator

potential_fn = generate_calculator(pme_generator.pme_force.get_energy, pme_generator)
print(potential_fn)

<function generate_calculator.<locals>.admp_calculator at 0x7fd6c02f2950>


Note in here, we use a function called `compute_leading_terms` to compute the atomic charges, and feed it into `pme_generator.pme_force.get_energy`, so geometry-dependent charge fluctuations are realized.

Let's see how the wrapped `potential_fn` works:

First, prepare the inputs

In [9]:
positions = jnp.array(pdb.positions._value) * 10
a, b, c = pdb.topology.getPeriodicBoxVectors()
box = jnp.array([a._value, b._value, c._value]) * 10

Consturct neighbor list 

In [10]:
import dmff.common.nblist as nblist
nbl = nblist.NeighborList(box, rc)
pairs = nbl.allocate(positions).idx.T
print(pairs)

[[0 1]
 [0 2]
 [1 2]
 [0 3]
 [1 3]
 [2 3]
 [0 4]
 [1 4]
 [2 4]
 [3 4]
 [0 5]
 [1 5]
 [2 5]
 [3 5]
 [4 5]
 [6 6]
 [6 6]
 [6 6]]


Call the potential function

In [11]:
E = potential_fn(positions, box, pairs)
print(E)

-41.352478


You can use the auto differentiation function in jax to compute force:

In [12]:
force_fn = jax.grad(potential_fn, argnums=(0))
F = force_fn(positions, box, pairs)
print(F)

[[  7.6667786 -11.7369795   7.985573 ]
 [-75.11585    58.206787   25.156063 ]
 [  1.8800697   4.9668303 -14.624431 ]
 [ 67.59385   -38.273987  -20.533588 ]
 [  2.5327225   5.2766438  -4.1624002]
 [ -4.5456085 -18.439816    6.118574 ]]
